In [0]:
spark.sql("USE CATALOG hp_az_catalog")
spark.sql("USE SCHEMA hp_iris_ml")

In [0]:
%pip install mlflow=='3.4.0'
dbutils.library.restartPython()

In [0]:
import pandas as pd
from sklearn import datasets
import mlflow
from mlflow.pyfunc import load_model

In [0]:
import mlflow
import pandas as pd
from datetime import datetime
from sklearn.datasets import load_iris

# Load model from registry
model_name = "irisclassifierdemo"
model_uri = f"models:/{model_name}@champion"
model = load_model(model_uri)

# Load or simulate new data
iris = load_iris()
X = pd.DataFrame(iris.data, columns=iris.feature_names)
print(X)
# Predict
preds = model.predict(X)
pred_df = X.copy()
pred_df["prediction"] = preds
pred_df["timestamp"] = datetime.now()
pred_df.columns = (
    pred_df.columns
    .str.replace(" ", "_", regex=False)
    .str.replace("(", "", regex=False)
    .str.replace(")", "", regex=False)
)
print(pred_df.head())
df_spark = spark.createDataFrame(pred_df)
df_spark.write.mode("append").format("delta").saveAsTable("iris_predictions")
print("Inference complete and saved to iris_predictions")